In [7]:
import librosa
import numpy as np
import parselmouth
import pandas as pd
import os

In [ ]:
def extract_features(file_path, mood_label=None):
    signal, sample_rate = librosa.load(file_path, sr=None)
    snd = parselmouth.Sound(file_path)

    # Pitch
    pitches, magnitudes = librosa.piptrack(y=signal, sr=sample_rate)
    pitch_values = pitches[magnitudes > np.median(magnitudes)]
    pitch_mean = np.mean(pitch_values) if len(pitch_values) > 0 else 0
    pitch_std = np.std(pitch_values) if len(pitch_values) > 0 else 0

    # Jitter & Shimmer
    point_process = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 500)
    jitter_local = parselmouth.praat.call(point_process, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    shimmer_local = parselmouth.praat.call([snd, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

    # HNR
    hnr = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr_value = parselmouth.praat.call(hnr, "Get mean", 0, 0)

    # Energia
    rms_energy = librosa.feature.rms(y=signal)[0]
    energy_mean = np.mean(rms_energy)
    energy_std = np.std(rms_energy)

    # MFCCs
    mfcc = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)

    # Speech rate
    onset_env = librosa.onset.onset_strength(y=signal, sr=sample_rate)
    _, beat_frames = librosa.beat.beat_track(onset_envelope=onset_env, sr=sample_rate)
    syllable_count = len(beat_frames)
    duration_sec = librosa.get_duration(y=signal, sr=sample_rate)
    speech_rate = syllable_count / duration_sec if duration_sec > 0 else 0

    # Zero-Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y=signal)[0]
    zcr_mean = np.mean(zcr)

    # Spectral features
    spectral_centroid = librosa.feature.spectral_centroid(y=signal, sr=sample_rate)[0]
    spectral_flux = np.mean(np.diff(spectral_centroid))
    spectral_rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sample_rate)[0]
    rolloff_mean = np.mean(spectral_rolloff)

    feature_dict = {
        'File': os.path.basename(file_path),
        'Speech Duration': duration_sec,
        'Pitch': pitch_mean,
        'Pitch Std': pitch_std,
        'Speech Rate': speech_rate,
        'Jitter': jitter_local,
        'Shimmer': shimmer_local,
        'Energy Mean': energy_mean,
        'Energy Std': energy_std,
        'HNR': hnr_value,
        'ZCR': zcr_mean,
        'Spectral Flux': spectral_flux,
        'Spectral Centroid Mean': np.mean(spectral_centroid),
        'Spectral Rolloff Mean': rolloff_mean,
        'Mood': mood_label if mood_label is not None else 'unknown'
    }

    for i in range(13):
        feature_dict[f'mfcc_{i}'] = mfcc_mean[i]

    return feature_dict

def batch_extract_from_main_folder(main_folder, output_csv='features.csv'):
    all_features = []


    for mood_label in os.listdir(main_folder):
        mood_path = os.path.join(main_folder, mood_label)
        if not os.path.isdir(mood_path):
            continue  


        for filename in os.listdir(mood_path):
            if not filename.lower().endswith(('.wav')):
                continue

            file_path = os.path.join(mood_path, filename)
            try:
                features = extract_features(file_path, mood_label=mood_label)
                all_features.append(features)
            except Exception as e:
                print(f"Erro ao processar {file_path}: {e}")

    # Salvar tudo num CSV
    df = pd.DataFrame(all_features)
    df.to_csv(output_csv, index=False)
    print(f"Todos os features foram salvos em: {output_csv}")



In [9]:
batch_extract_from_main_folder("../Emotions_Simple/")

Todos os features foram salvos em: features.csv
